In [1]:
import requests
import datetime
import json

def 시간변환(년: int, 월: int, 일: int, 시: int, 분):
    input_time = datetime.datetime(년, 월, 일, 시, 분)
    return input_time.strftime("%Y-%m-%dT%H:%M:%S%z+09:00")

# 상품생성 프로세스에 필요한 값 설정

### 설정 값
- 환경 (dev, test)
- 이메일, 비밀번호
- 클래스 아이디
- 수강기간
- 정가, 할인가
- 시작시간, 종료시간
- 상품상태( 공개, 비공개 )

In [2]:
환경 = "dev"

이메일, 비밀번호 = "taekyun@quriously.com", "xordbs224!"

클래스아이디 = 417

# 수강 기간은 일단 일 단위로 기재하셔야 합니다.
수강기간 = 211
정가, 할인가 = 1000000, 700000

# 시작시간은 무조건 필수! 한국 시간으로 기재하셔야 합니다.
# 종료 시간이 필요하면 시작 시간과 동일한 형태로 진행하면 됩니다.
# ex) 시작시간, 종료시간 = 시간_변환(년=2023, 월=7, 일=12, 시=12, 분=0), None
# ex) 시작시간, 종료시간 = 시간변환(년=2023, 월=7, 일=12, 시=12, 분=0), 시간변환(년=2023, 월=8, 일=12, 시=12, 분=0)
시작시간, 종료시간 = 시간변환(년=2023, 월=7, 일=12, 시=12, 분=0), None

# 상태의 종류는 비공개(UNLISTED), 공개(LISTED)
상품상태 = "LISTED"

# 초기 설정
### 로그인 및 토큰 발행

In [3]:
account_login_url = f"https://quriously-{환경}.airklass.biz/rest/account/login"

account_headers = {"Authorization": "None"}
login_data = {
    "type": "EMAIL",
    "value": 이메일,
    "password": 비밀번호,
}
response = requests.post(account_login_url, headers=account_headers, data=json.dumps(login_data))
access_token = response.json().get("token").get("access")

base_url = f"https://quriously-{환경}.airklass.biz/rest/commerce"
headers = {"Authorization": f"Qube-token {access_token}"}

print("로그인 및 토큰 발행 성공")

로그인 및 토큰 발행 성공


# 상품 생성

In [4]:
path = "/product/klass-ticket"
url = base_url + path

data = {"klassId": 클래스아이디}

response = requests.post(url, headers=headers, data=json.dumps(data))

상품키 = None
if response.status_code == 200:
    상품키 = response.text
elif response.status_code == 400:
    response_data = response.json()
    상품키 = response_data.get("data").get("productKey")
else:
    raise Exception("상품_생성_실패")

print("상품 생성 성공")

상품 생성 성공


# 상품 아이템 생성

In [5]:
path = f"/product/{상품키}/option-group/_PERIOD/option-item"
url = base_url + path

data = {"label": f"{수강기간}일", "value": f"{수강기간}d"}

response = requests.post(url, headers=headers, data=json.dumps(data))
response_data = response.json()

옵션아이템키 = response_data.get("id")

print("상품 아이템 생성 성공")

상품 아이템 생성 성공


# 상품 유형 생성

In [6]:
path = f"/product/{상품키}/variant"
url = base_url + path

data = {"platform": "web", "optionIdMap": {"_PERIOD": 옵션아이템키}}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code != 200:
    raise Exception("상품_유형_생성_실패")
상품유형키 = response.text

print("상품 유형 생성 성공")

상품 유형 생성 성공


# 상품 유형 가격 생성

In [7]:
path = f"/product/{상품키}/variant/{상품유형키}/product-price"
url = base_url + path

data = {
    "listPrice": 정가,
    "salesPrice": 할인가,
    "start": 시작시간,
    "end": 종료시간
}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code != 201:
    raise Exception("상품_유형_가격_생성_실패")

print("상품 유형 가격 생성 성공")

상품 유형 가격 생성 성공


# 상품의 상태 변경

In [8]:
path = f"/product/{상품키}/variant/{상품유형키}/status"
url = base_url + path

data = {"nextStatus": 상품상태}
response = requests.patch(url, headers=headers, data=json.dumps(data))
if response.status_code != 200:
    raise Exception("상품의_상태_변경_실패")

print("상품의 상태 변경 성공")

상품의 상태 변경 성공
